# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# Put csv into a dataframe
filepath = '../WeatherPy/output_data/cities1.csv'
city_df = pd.read_csv(filepath)

In [11]:
city_df = city_df[["City", "Longitude", "Latitude", "Temperature", "Humidity", "Cloudiness", "Wind Speed"]]
city_df.head(10)

,City,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed
0,ponta do sol,-17.10,32.67,71.492,43,20,9.80
1,nanortalik,-45.24,60.14,45.752,90,100,3.06
2,albany,-73.97,42.60,81.302,60,33,1.08
3,ushuaia,-68.30,-54.80,32.270,93,90,5.70
4,punta arenas,-70.92,-53.15,37.472,88,40,6.20
5,rikitea,-134.97,-23.12,68.504,71,17,7.06
6,new norfolk,147.06,-42.78,43.070,86,100,1.06
7,dinan,-2.25,48.42,63.356,88,86,3.10
8,ketchikan,-131.65,55.34,59.972,77,90,4.10
9,barrow,-156.79,71.29,35.870,80,90,6.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [13]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_df[["Latitude", "Longitude"]].astype(float)

# Use Humidity as the weight.
humidity = city_df["Humidity"].astype(float)


In [15]:
# Add Heatmap layer to map.
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# Drop null values
city_df = city_df.dropna()

# Narrow down my ideal conditions
ideal_city_df = city_df.loc[ (city_df["Temperature"] <= 80) & 
                             (city_df["Temperature"] >= 68) &
                             (city_df["Wind Speed"] < 5) & 
                             (city_df["Cloudiness"] == 0) & 
                             (city_df["Humidity"] < 70), : ]
print(f'Rows in ideal dataframe: {ideal_city_df.shape[0]}')
ideal_city_df.head(14)

Rows in ideal dataframe: 14


,City,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed
31,nalut,10.85,30.33,77.036,23,0,4.93
74,hami,93.45,42.80,77.954,33,0,2.65
117,walvis bay,14.51,-22.96,68.324,14,0,4.10
118,beyneu,55.20,45.32,74.966,47,0,2.60
130,guararapes,-50.64,-21.26,69.764,34,0,2.39
177,syracuse,15.28,37.09,75.632,64,0,2.60
201,katherine,132.27,-14.47,68.270,49,0,2.60
226,jiayuguan,98.30,39.82,69.170,41,0,3.30
229,kerman,57.08,30.28,71.870,18,0,4.10
274,luderitz,15.16,-26.65,70.070,22,0,3.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
# Build the hotel dataframe
hotel_df = ideal_city_df[["City", "Longitude", "Latitude", "Temperature", "Humidity", "Cloudiness", "Wind Speed"]]
hotel_df["Hotel"] = ''


In [18]:
# The Longitude latitude values for the first row, for whatever reason,
# are not specific enough to provide a result - so I am going to update 
# them manually
hotel_df.at[31, "Longitude"] = 10.9427517
hotel_df.at[31, "Latitude"] = 31.8666867
hotel_df.head(14)

,City,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel
31,nalut,10.942752,31.866687,77.036,23,0,4.93,
74,hami,93.450000,42.800000,77.954,33,0,2.65,
117,walvis bay,14.510000,-22.960000,68.324,14,0,4.10,
118,beyneu,55.200000,45.320000,74.966,47,0,2.60,
130,guararapes,-50.640000,-21.260000,69.764,34,0,2.39,
177,syracuse,15.280000,37.090000,75.632,64,0,2.60,
201,katherine,132.270000,-14.470000,68.270,49,0,2.60,
226,jiayuguan,98.300000,39.820000,69.170,41,0,3.30,
229,kerman,57.080000,30.280000,71.870,18,0,4.10,
274,luderitz,15.160000,-26.650000,70.070,22,0,3.60,


In [ ]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.
radius = 5000
hotel_full_data = []


for i, row in hotel_df.iterrows():
    
    url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=formatted_address,name,opening_hours,rating&locationbias=circle:{radius}@{row["Latitude"]},{row["Longitude"]}&key={g_key}'
    response_json = requests.get(url).json()
    hotel_full_data.append(response_json)
    hotel_name = response_json["candidates"][0]["name"]
    hotel_df.at[i, "Hotel"] = hotel_name
    
    time.sleep(1)

hotel_df.head(14)

In [ ]:
# Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
